# arUSD trading volume by wallet

## How to start
You need to register an account with [Bitquery](https://graphql.bitquery.io/ide). You get your personal API key which you need to run this script.

Click on Play repeatedly until you reach the section where you get asked to enter your Bitquery API key. Enter the key and press play again. In a few seconds you will get the result.

Alternatively you can run this notebook locally and 

## Data returned
It is assumed that the address swapping is is also swapping out. Synth-to-synth swaps and transfers of synths between wallets are not considered.

arUSD contract: 0x025AB35fF6AbccA56d57475249baaEae08419039

Swapping contract: 0x948011e8ca8df1e9c83fee88967a5fc30c7a604b

Fee holder contract: 0xc768915863a333db9bb871bd687dd0a0ae41a3be

In [3]:
import os
from pathlib import Path
#import scripts.utils as utils
import utils
import pandas as pd
#from IPython.display import display, HTML
import matplotlib.pyplot as plt
#%matplotlib inline
from IPython.display import HTML
# pd.option_context('display.precision', 3)
# pd.options.display.float_format = '{:,.5e}'.format
api_key = None
if not os.path.isfile('.env'):
	api_key = input("Please enter your Bitquery API key. You can get one at https://graphql.bitquery.io/ide and paste it here: ")

start_date = "2022-06-01"

query_variables = {"start_date": start_date}

# Get all swap-out transactions
query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:100000
				desc: ["amount"]
			}
			success: true
			date: {since: $start_date}
			receiver:{notIn:["0xc768915863a333db9bb871bd687dd0a0ae41a3be","0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"]}
			sender:{is:"0x0000000000000000000000000000000000000000"}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		receiver {address}
		amount
		currency {symbol, address}
   }
  }
}
"""

swap_out_result = utils.bitqueryAPICall(query, query_variables, api_key)

swap_out = []
for r in swap_out_result["data"]["ethereum"]["transfers"]:
    swap_out.append([r["receiver"]["address"], r["amount"]])

# Get all swap-in transactions
query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:100000
				desc: ["amount"]
			}
			success: true
			date: {since: $start_date}
			receiver:{is:"0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"}
			sender:{notIn:["0x0000000000000000000000000000000000000000", "0xc768915863a333db9bb871bd687dd0a0ae41a3be"]}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		sender {address}
		amount
		currency {symbol, address}
   }
  }
}
"""

swap_in_result = utils.bitqueryAPICall(query, query_variables, api_key)

swap_in = []
for r in swap_in_result["data"]["ethereum"]["transfers"]:
    swap_in.append([r["sender"]["address"], r["amount"]])

# Get last transaction date
query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:1
				desc: ["block.timestamp.iso8601"]
			}
			success: true
			date: {since: $start_date}
			receiver:{notIn:["0xc768915863a333db9bb871bd687dd0a0ae41a3be","0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"]}
			sender:{is:"0x0000000000000000000000000000000000000000"}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		block {timestamp {iso8601}}
   }
  }
}
"""
last_out = utils.bitqueryAPICall(query, query_variables, api_key)

query = """
query ($start_date: ISO8601DateTime) {
ethereum(network: avalanche) {
    transfers(
			options: {
				limit:1
				desc: ["block.timestamp.iso8601"]
			}
			success: true
			date: {since: $start_date}
			receiver:{is:"0x948011e8ca8df1e9c83fee88967a5fc30c7a604b"}
			sender:{notIn:["0x0000000000000000000000000000000000000000", "0xc768915863a333db9bb871bd687dd0a0ae41a3be"]}
			currency: {is: "0x025AB35fF6AbccA56d57475249baaEae08419039"}
		) {
		block {timestamp {iso8601}}
   }
  }
}
"""
last_in = utils.bitqueryAPICall(query, query_variables, api_key)
last_tx_date=max(last_in["data"]["ethereum"]["transfers"][0]["block"]["timestamp"]["iso8601"],last_out["data"]["ethereum"]["transfers"][0]["block"]["timestamp"]["iso8601"])

# Filter out arUSD swap-out transactions from wallets that never performed a swap-in
df_out = utils.df(swap_out, columns=["address", "amount_out"])
df_in = utils.df(swap_in, columns=["address", "amount_in"])
int_df = pd.merge(df_out, df_in, how='inner', on=['address'])

# sum of amount_in and amount_out
int_df['amount'] = int_df['amount_in'] + int_df['amount_out']

# Output
print("Most recent transaction: "+last_tx_date)

# Sum of all swap-in and swap-out transactions
print("Total volume: "+str(int_df['amount'].sum().round(2)))


int_df.round(2)

# display(int_df)


Most recent transaction: 2022-10-01T18:56:40Z
Total volume: 4940108.76


,address,amount_out,amount_in,amount
0,0x02aceac337c6039eb9b0d2e43b730241ca620f73,1255268.75,1245479.93,2500748.68
1,0x42326781b4776719ec87d57246d19c79ea1f7629,699941.33,692595.14,1392536.48
2,0xd939c534bbb89463b1bef66d9201f4b287352e6d,258584.93,249601.08,508186.01
3,0xc5e297e508364c9b247a4b6209c5da73d9880867,86483.28,83388.42,169871.70
4,0x16a522067b5d88a803dd43f64dea23d28a5d3cc9,35788.73,735.00,36523.73
5,0xb7113ce37c448a4ebf5f94c1e360f3dd52d35729,31485.40,3625.57,35110.98
6,0xd35f615565b16b46f09c3e96553d175483978b73,31393.66,29839.40,61233.06
7,0x531563f28b6d595583a9158475d1048372a03e2c,30844.36,27976.49,58820.86
8,0x364705913b668b17f7595fba78b4e8888f5927f2,26245.99,7372.21,33618.20
9,0x274aaa262e11cac18865a3e8d0cd43675d0a4fe1,25892.18,22194.44,48086.62
